<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark pyngrok

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = (
    SparkSession
    .builder
    .appName("halltape_pyspark_local")
    .getOrCreate()
)

print("Активные Spark сессии:", spark.sparkContext.uiWebUrl)

In [ ]:
from pyngrok import ngrok, conf

# Прочитаем токен из файла token.txt
with open("/content/drive/MyDrive/Colab Notebooks/py_spark/data/token.txt") as f:
    auth_token = f.read().strip()

import os
os.environ["NGROK_AUTHTOKEN"] = auth_token

# Проверим, что токен установлен
print("Токен успешно добавлен в переменные окружения.")
public_url = ngrok.connect(4040, "http")
print(public_url)

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/py_spark/data/customs_data.csv'

In [ ]:
spark.read.csv(PATH).show()

In [ ]:
df = spark.read.csv(PATH, sep=';', header=True)

df.show(truncate=False)

# df.show(2, False, True)

In [ ]:
result = (
    df
    .withColumnRenamed("direction_eng", "direction")
    .withColumnRenamed("measure_eng", "measure")
)

result.columns

In [ ]:
result.select('country').distinct().show(10, truncate=False)

In [ ]:
# Два варианта написании фильтрации
df_de = (
    result
    .where(F.col('country') == 'DE')
    .where(F.col('value').isNotNull())
)

In [ ]:
df_de.show(2, truncate=False)

In [ ]:
final = (
    df_de
    .select(
        'month',
        'country',
        'code',
        'value',
        'netto',
        'quantity',
        'region',
        'district',
        'direction',
        'measure',
        F.col('load_date').cast('date'),
    )
)

final.show(2, truncate=False)

In [ ]:
(
    final
    .coalesce(1)
    .write
    .format('csv')
    .options(header='True', sep=';')
    .csv('data/final_one_file')
)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CollectExample").getOrCreate()

# Создаем DataFrame
df = spark.createDataFrame([
    ("Alice", 25, "London"),
    ("Bob", 30, "New York"),
    ("Charlie", 35, "Paris"),
    ("Diana", 28, "Tokyo")
], ["name", "age", "city"])

# collect() возвращает список Row объектов
all_data = df.collect()
print(f"Тип: {type(all_data)}")  # <class 'list'>
print(f"Размер: {len(all_data)}")  # 4



In [ ]:
type(df)

In [ ]:
# df.collect()

In [ ]:
gf = df.repartition(1)

In [ ]:
gf.show()

In [ ]:
df.show()